In [9]:
import yaml
import pathlib
from typing import Any, List, Tuple

# Path to the configuration file
config_path = pathlib.Path('../../config/config.yaml')

# Load the YAML file
with config_path.open('r') as f:
    cfg = yaml.safe_load(f)

# Pretty‑print ------------------------------------------------------------
pretty_yaml = yaml.safe_dump(
    cfg,
    default_flow_style=False,   # forces block style (one key per line)
    sort_keys=False,            # keep the order from the file
    allow_unicode=True,         # keep any non‑ASCII characters
    indent=2                    # nice 2‑space indentation
)
print(pretty_yaml)


project:
  name: AWS-CapacityForecaster
  version: 1.0
  description: Cloud-native enterprise server capacity forecasting system
aws:
  region: us-east-1
  profile: default
  bucket_name: sean-capacity-forecast-data
  raw_prefix: raw/server_metrics/
  processed_prefix: processed/server_metrics/
  features_prefix: processed/features/
  sagemaker_role_arn: ''
  sagemaker_instance_type: ml.t3.medium
  athena_database: capacity_forecaster_db
  athena_workgroup: primary
data:
  num_servers: 120
  years_of_data: 4
  granularity: daily
  start_date: '2022-01-01'
  end_date: '2025-12-31'
  seasonality:
    weekly: true
    quarterly_peaks: true
    holiday_effect: true
  p95_ranges:
    cpu:
    - 10.0
    - 95.0
    memory:
    - 15.0
    - 92.0
    disk:
    - 5.0
    - 85.0
    network_in:
    - 20.0
    - 500.0
    network_out:
    - 10.0
    - 300.0
ml:
  forecast_horizon_months: 6
  models:
  - name: Prophet
    enabled: true
    params:
      yearly_seasonality: true
      weekly_season

In [10]:
# ----------------------------------------------------------------------
# Helper: safely fetch a nested value given a list of keys
# ----------------------------------------------------------------------
def get_nested(data: dict, keys: List[str], default: Any = None) -> Any:
    """
    Walks through ``data`` following ``keys``.
    Returns ``default`` if any intermediate key is missing.
    """
    cur = data
    for k in keys:
        if isinstance(cur, dict) and k in cur:
            cur = cur[k]
        else:
            return default
    return cur

# ----------------------------------------------------------------------
# 1️⃣  AWS region (top‑level key)
# ----------------------------------------------------------------------
aws_region = get_nested(cfg, ["aws", "region"], "us-east-1")
print(f"AWS region: {aws_region}")

# ----------------------------------------------------------------------
# 2️⃣  Forecast horizon (months) – under the `ml` section
# ----------------------------------------------------------------------
forecast_horizon = get_nested(cfg, ["ml", "forecast_horizon_months"], 0)
print(f"Forecast horizon (months): {forecast_horizon}")

# ----------------------------------------------------------------------
# 3️⃣  List of enabled ML models and a quick view of their params
# ----------------------------------------------------------------------
enabled_models: List[Tuple[str, dict]] = []
for model in cfg.get("ml", {}).get("models", []):
    if model.get("enabled"):
        enabled_models.append((model["name"], model.get("params", {})))

print("\nEnabled ML models and their parameters:")
for name, params in enabled_models:
    print(f" • {name}")
    for p_key, p_val in params.items():
        print(f"     - {p_key}: {p_val}")

# ----------------------------------------------------------------------
# 4️⃣  Example: pull a specific parameter from a given model
# ----------------------------------------------------------------------
def get_model_param(model_name: str, param_key: str, default: Any = None) -> Any:
    """
    Returns the value of ``param_key`` for the model named ``model_name``.
    """
    for model in cfg.get("ml", {}).get("models", []):
        if model.get("name") == model_name:
            return model.get("params", {}).get(param_key, default)
    return default

# Example – get Prophet's yearly_seasonality flag
prophet_yearly = get_model_param("Prophet", "yearly_seasonality", False)
print(f"\nProphet yearly_seasonality: {prophet_yearly}")

# ----------------------------------------------------------------------
# 5️⃣  Generic usage – fetch any nested key you need
# ----------------------------------------------------------------------
# Example: get the default logging format
log_fmt = get_nested(cfg, ["logging", "format"], "<no format>")
print(f"\nLogging format: {log_fmt}")

AWS region: us-east-1
Forecast horizon (months): 6

Enabled ML models and their parameters:
 • Prophet
     - yearly_seasonality: True
     - weekly_seasonality: True
     - daily_seasonality: False
     - changepoint_prior_scale: 0.05
 • RandomForest
     - n_estimators: 200
     - max_depth: 10
     - random_state: 42
 • GradientBoosting
     - n_estimators: 150
     - learning_rate: 0.05
     - max_depth: 6

Prophet yearly_seasonality: True

Logging format: %(asctime)s - %(name)s - %(levelname)s - %(message)s
